<a href="https://colab.research.google.com/github/LINA-LY/AI/blob/main/MLOps_Pipeline_Case_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Building an MLOps Pipeline using Apache Airflow

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

#lead the dataset
data = pd.read_csv('/content/screentime_analysis.csv')

#check for missing values and duplicates
print(data.isnull().sum())
print(data.duplicated().sum())

#convert data column to datatime and extract feartures
data['Date'] = pd.to_datetime(data['Date'])
data['DayOfWeek'] = data['Date'].dt.dayofweek
data['Month'] = data['Date'].dt.month

#encode the categorical 'App' column using one_hot encoding
data = pd.get_dummies(data, columns=['App'], drop_first=True)

#scale numerical features using MinMaxScaler
scaler = MinMaxScaler()
data[['Notifications' , 'Times Opened']] = scaler.fit_transform(data[['Notifications', 'Times Opened']])

#feature enginneering
data['Previous_Day_Usage'] = data['Usage (minutes)'].shift(1)
data['Notifications_x_TimesOpened'] = data['Notifications'] * data['Times Opened']

#save the preprocessed data to file
data.to_csv('preprocessed_screentime_analysis.csv', index=False)

Date               0
App                0
Usage (minutes)    0
Notifications      0
Times Opened       0
dtype: int64
0


**Training the Model**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

#split data into features and target variable
X = data.drop(columns=['Usage (minutes)', 'Date'])
y = data['Usage (minutes)']

#tain-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#train the model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

#evaluate the model
predictions = model.predict(X_test)
mae  = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 15.398500000000002


**Now, we will define the DAG and task to build the pipeline:**

In [ ]:
pip install apache-airflow

In [ ]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime

#define the data preprocessing function
def proprecess_data():
  file_path = '/content/screentime_analysis.csv'
  data = pd.read_csv(file_path)

  data['Data'] = pd.to_datetime(data['Date'])
  data['DayOfWeek'] = data['Date'].dt.dayofweek
  data['Month'] = data['Date'].dt.month

  data = data.drop(columns=['Date'])

  data = pd.get_dummies(data, columns=['App'], drop_first=True)

  scaler = MinMaxScaler()
  data[['Notifications', 'Times Opened']] = scaler.fit_transform(data[['Notifications', 'Times Opened']])
  preprocessed_path = '/content/preprocessed_screentime_analysis.csv'
  data.to_csv(preprocessed_path, index= False )
  print(f"preprocessed data saved to {preprocessed_path}")


#define the DAG
dag = DAG(
    dag_id='data_preprecessing',
    schedule_interval='@daily',
    start_date=datetime(2025,1,1),
    catchup=False,
)

#define the task
preprocess_task = PythonOperator(
    task_id= 'preprocess',
    python_callable = proprecess_data,
    dag=dag,
)

<ipython-input-9-64f36bbd1451>:26: RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead. 
  dag = DAG(


In [ ]:
!pip install apache-airflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.4/548.4 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.6 MB/s eta 0:00:00
   ━━━━

In [ ]:
!airflow db init

DB: sqlite:////root/airflow/airflow.db
[2025-04-13T15:37:47.390+0000] {migration.py:207} INFO - Context impl SQLiteImpl.
[2025-04-13T15:37:47.392+0000] {migration.py:210} INFO - Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running stamp_revision  -> 5f2621c13b39
WARNI [airflow.models.crypto] empty cryptography key - values will not be stored encrypted.
Initialization done


In [ ]:
!airflow webserver --port 8080

[2025-04-13T15:38:47.674+0000] {configuration.py:2112} INFO - Creating new FAB webserver config file in: /root/airflow/webserver_config.py
  ____________       _____________
 ____    |__( )_________  __/__  /________      __
____  /| |_  /__  ___/_  /_ __  /_  __ \_ | /| / /
___  ___ |  / _  /   _  __/ _  / / /_/ /_ |/ |/ /
 _/_/  |_/_/  /_/    /_/    /_/  \____/____/|__/
Running the Gunicorn Server with:
Workers: 4 sync
Host: 0.0.0.0:8080
Timeout: 120
Logfiles: - -
Access Logformat: 
/usr/local/lib/python3.11/dist-packages/flask_limiter/extension.py:324 UserWarning: Using the in-memory storage for tracking rate limits as no storage was explicitly specified. This is not recommended for production use. See: https://flask-limiter.readthedocs.io#configuring-a-storage-backend for documentation about configuring the storage backend.
[2025-04-13T15:38:52.588+0000] {override.py:1526} INFO - Inserted Role: Admin
[2025-04-13T15:38:52.595+0000] {override.py:983} WARNING - No user yet created, us

In [ ]:
!airflow scheduler

  ____________       _____________
 ____    |__( )_________  __/__  /________      __
____  /| |_  /__  ___/_  /_ __  /_  __ \_ | /| / /
___  ___ |  / _  /   _  __/ _  / / /_/ /_ |/ |/ /
 _/_/  |_/_/  /_/    /_/    /_/  \____/____/|__/
[2025-04-13T15:41:14.712+0000] {utils.py:162} INFO - NumExpr defaulting to 2 threads.
[2025-04-13T15:41:14.879+0000] {executor_loader.py:258} INFO - Loaded executor: SequentialExecutor
[2025-04-13 15:41:15 +0000] [2179] [INFO] Starting gunicorn 23.0.0
[2025-04-13 15:41:15 +0000] [2179] [INFO] Listening at: http://[::]:8793 (2179)
[2025-04-13 15:41:15 +0000] [2179] [INFO] Using worker: sync
[2025-04-13T15:41:15.040+0000] {scheduler_job_runner.py:950} INFO - Starting the scheduler
[2025-04-13T15:41:15.041+0000] {scheduler_job_runner.py:957} INFO - Processing each file at most -1 times
[2025-04-13 15:41:15 +0000] [2180] [INFO] Booting worker with pid: 2180
[2025-04-13T15:41:15.053+0000] {manager.py:174} INFO - Launched DagFileProcessorManager with pid: 2181